In [1]:
%cd /home/ubuntu/Qwen-Image-Edit-Angles

/home/ubuntu/Qwen-Image-Edit-Angles


In [2]:
import pandas as pd
from matplotlib import pyplot as plt

from qwenimage.experiments.experiments_qwen import PipeInputs

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Skipping import of cpp extensions due to incompatible torch version 2.9.1+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
TMA benchmarks will be running without grid constant TMA descriptor.
2025-11-15 00:25:52.163576: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the envi

In [3]:
from qwenimage.experiment import ExperimentConfig
from qwenimage.experiments.experiments_qwen import ExperimentRegistry


experiment_names = [
    "qwen_base",
    "qwen_lightning_fa3_aot_int8_fuse_4step_fbcache_055_downsize512",
    "qwen_lightning_fa3_aot_int8_fuse_1step_fbcache_055_downsize512",
]

report_dir = ExperimentConfig().report_dir

experiment_outputs = {}
num_outputs = {}
for name in experiment_names:
    output_dir = report_dir / f"{name}_outputs"
    all_output_paths = sorted(list(output_dir.glob("*.jpg")))
    experiment_outputs[name] = all_output_paths
    num_outputs[name] = len(all_output_paths)




In [4]:
experiment_outputs["qwen_base"][0]

PosixPath('reports/qwen_base_outputs/000.jpg')

In [5]:
import math
import textwrap
from pathlib import Path

import matplotlib.font_manager as fm
import numpy as np
from PIL import Image

# Configure Chinese font
font_path = Path("scripts/assets/STSong.ttf")
chinese_font = fm.FontProperties(fname=str(font_path))

comparison_dir = report_dir / "comparison_grid_4"
comparison_dir.mkdir(exist_ok=True, parents=True)

# Create PipeInputs instance to access original images and prompts
pipe_inputs = PipeInputs(seed=42)

comparable_outputs = min(num_outputs.values())
num_experiments = len(experiment_names)

# For each output index, create a comparison grid
for idx in range(comparable_outputs):
    # Get the original image and prompt for this index
    inputs = pipe_inputs[idx]
    original_image = inputs["image"][0]
    prompt = inputs["prompt"]
    
    # Calculate grid size: +1 for original image
    total_plots = num_experiments + 1
    cols = int(math.ceil(math.sqrt(total_plots)))
    if total_plots == 3:
        cols = 3 # hard override for formatting
    rows = int(math.ceil(total_plots / cols))
    
    fig, axes = plt.subplots(rows, cols, figsize=(6 * cols, 6 * rows))
    
    # Flatten axes array for easier indexing
    if total_plots == 1:
        axes = np.array([axes])
    else:
        axes = axes.flatten() if isinstance(axes, np.ndarray) else np.array([axes])
    
    # Plot original image in the first subplot
    ax = axes[0]
    ax.imshow(original_image)
    ax.set_title("Original Image", fontsize=14, fontweight='bold', color='blue')
    ax.axis('off')
    
    # Plot each experiment's output for this index
    for exp_idx, exp_name in enumerate(experiment_names):
        ax = axes[exp_idx + 1]  # +1 because first subplot is original image
        
        # Check if this experiment has an output at this index
        if idx < num_outputs[exp_name]:
            img_path = experiment_outputs[exp_name][idx]
            img = Image.open(img_path)
            ax.imshow(img)
            ax.set_title(exp_name, fontsize=14, fontweight='bold')
        else:
            # No output for this experiment at this index
            ax.text(0.5, 0.5, 'N/A', ha='center', va='center', fontsize=20)
            ax.set_title(exp_name, fontsize=14, fontweight='bold', color='gray')
        
        ax.axis('off')
    
    # Hide any unused subplots
    for plot_idx in range(total_plots, len(axes)):
        axes[plot_idx].axis('off')
    
    # Wrap prompt text for better display
    wrapped_prompt = '\n'.join(textwrap.wrap(prompt, width=100))
    
    # Add a main title with the prompt
    fig.suptitle(
        f'Output Comparison - Index {idx:03d}\nPrompt: {wrapped_prompt}',
        fontsize=16,
        fontweight='bold',
        y=0.99,
        fontproperties=chinese_font
    )
    
    plt.tight_layout()
    
    # Save the figure
    output_path = comparison_dir / f"comparison_{idx:03d}.jpg"
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close(fig)
    
    print(f"Saved comparison grid {idx + 1}/{comparable_outputs} to {output_path}")

print(f"\nAll comparison grids saved to {comparison_dir}")


720 input combinations


Saved comparison grid 1/32 to reports/comparison_grid_4/comparison_000.jpg
Saved comparison grid 2/32 to reports/comparison_grid_4/comparison_001.jpg
Saved comparison grid 3/32 to reports/comparison_grid_4/comparison_002.jpg
Saved comparison grid 4/32 to reports/comparison_grid_4/comparison_003.jpg
Saved comparison grid 5/32 to reports/comparison_grid_4/comparison_004.jpg
Saved comparison grid 6/32 to reports/comparison_grid_4/comparison_005.jpg
Saved comparison grid 7/32 to reports/comparison_grid_4/comparison_006.jpg
Saved comparison grid 8/32 to reports/comparison_grid_4/comparison_007.jpg
Saved comparison grid 9/32 to reports/comparison_grid_4/comparison_008.jpg
Saved comparison grid 10/32 to reports/comparison_grid_4/comparison_009.jpg
Saved comparison grid 11/32 to reports/comparison_grid_4/comparison_010.jpg
Saved comparison grid 12/32 to reports/comparison_grid_4/comparison_011.jpg
Saved comparison grid 13/32 to reports/comparison_grid_4/comparison_012.jpg
Saved comparison grid